### pymupdf4llm만 사용

정성적 성능 비교를 위해 작성

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import textwrap
from copy import deepcopy
from pathlib import Path

import pymupdf4llm
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from pyzerox.core.types import ZeroxOutput

/Users/kyuyeon/opt/miniconda3/envs/msbs/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
DATASET_DIR = "dataset"
MODEL_NAME = "gpt-4o"
EMBEDDING_MODEL = "text-embedding-3-large"

In [4]:
def strip_prompt(prompt: str) -> str:
    return textwrap.dedent(prompt).strip()

In [5]:
pdf_paths = list(Path(DATASET_DIR).rglob("DATA*.pdf"))
pdf_paths = sorted([str(pdf_path) for pdf_path in pdf_paths])

In [6]:
pymu_results_gathered = [
    pymupdf4llm.to_markdown(pdf_path, page_chunks=True, show_progress=False)
    for pdf_path in pdf_paths
]

In [7]:
def get_splitter() -> MarkdownHeaderTextSplitter:
    _target_headers = [
        ("#", "#"),
        ("##", "##"),
        ("###", "###"),
        ("####", "####"),
        ("#####", "#####"),
        ("######", "######"),
    ]
    return MarkdownHeaderTextSplitter(headers_to_split_on=_target_headers, strip_headers=False)


def split_pages_into_document_bundles(result: list[dict] | ZeroxOutput) -> list[dict]:
    # Result of pymupdf4llm
    if isinstance(result, list):
        spl = get_splitter()
        doc_bundles = []
        for page in result:
            docs = spl.split_text(page["text"])
            for doc in docs:
                doc_bundles.append(
                    {
                        "file_name": Path(page["metadata"]["file_path"]).name,
                        "page_number": page["metadata"]["page"],
                        "document": doc,
                    }
                )
        print(f"pymupdf4llm: {len(result)} pages split into {len(doc_bundles)} document bundles.")
    # Result of pyzerox (OCR)
    elif isinstance(result, ZeroxOutput):
        spl = get_splitter()
        doc_bundles = []
        for page in result.pages:
            docs = spl.split_text(page.content)
            for doc in docs:
                doc_bundles.append(
                    {
                        "file_name": Path(result.file_name).name,
                        "page_number": page.page,
                        "document": doc,
                    }
                )
        print(f"pyzerox: {len(result.pages)} pages split into {len(doc_bundles)} document bundles.")
    else:
        raise ValueError("The type of argument 'result' must be either a dict or ZeroxOutput.")

    return doc_bundles


def prepend_info_to_documents(
    document_bundles: list[dict],
    prepend_file_name: bool = True,
    prepend_metadata: bool = True,
) -> list[Document]:
    # Return only the document from document_bundles
    if not prepend_file_name and not prepend_metadata:
        return [doc["document"] for doc in document_bundles]

    doc_bundles_ = deepcopy(document_bundles)
    if prepend_metadata:
        for doc_bundle in doc_bundles_:
            doc = doc_bundle["document"]
            for header, header_content in reversed(doc.metadata.items()):
                if header_content not in doc.page_content:
                    doc.page_content = f"{header} {header_content}\n{doc.page_content}"
    if prepend_file_name:
        for doc_bundle in doc_bundles_:
            doc = doc_bundle["document"]
            doc.page_content = (
                f"File name: '{doc_bundle['file_name']}'\n"
                f"Page number: {doc_bundle['page_number']}\n"
                f"{doc.page_content}"
            )
    return [doc["document"] for doc in doc_bundles_]

In [8]:
pymu_all_docs = []

for pymu_result in pymu_results_gathered:
    pymu_docs = split_pages_into_document_bundles(pymu_result)
    pymu_docs_with_info = prepend_info_to_documents(pymu_docs)
    pymu_all_docs += pymu_docs_with_info

pymupdf4llm: 54 pages split into 91 document bundles.
pymupdf4llm: 36 pages split into 42 document bundles.
pymupdf4llm: 2 pages split into 4 document bundles.
pymupdf4llm: 6 pages split into 7 document bundles.
pymupdf4llm: 6 pages split into 6 document bundles.
pymupdf4llm: 13 pages split into 19 document bundles.
pymupdf4llm: 1 pages split into 1 document bundles.


In [9]:
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

vector_store_txt = FAISS.from_documents(
    documents=pymu_all_docs,
    embedding=embeddings,
)

In [10]:
RETRIEVE_K = 8

retriever_txt = vector_store_txt.as_retriever(search_kwargs={"k": RETRIEVE_K})

In [11]:
result = retriever_txt.invoke(
    "로봇과 비전간의 캘리브레이션을 어떻게 하는지 알려줘 그리고 캘리브레이션이 무엇인지도 설명해줘"
)
for d in result:
    print(d.page_content)
    break

File name: 'DATA2. e-F@ctory Model Line_Robot-Vision간 모델링 및 캘리브레이션 방법.pdf'
Page number: 4
**캘리브레이션이란? 카메라화면상의물체위치정보변화량과로봇의실제이동거리간의상관관계를정의하고, 카메라화면상에서1픽셀의차이가실제로봇에서는얼마나**
움직여야하는지정의한다.  
1픽셀  
0.026cm 1px  
0.026cm


In [12]:
prompt_text = """
    You are an assistant for answering questions based on equipment manuals.
    Use the following retrieved context to answer the question.
    If the answer is unclear, try to deduce it from the provided contexts, or state that you don't know.
    Avoid using the context if it appears irrelevant or deteriorated.
    Provide your answer in Korean and format it as Markdown if applicable.
    If you are unsure about the context, you can ask for the file name to refer to.

    ## Context:
    {context_txt}

    ## Question:
    {question}

    ## Answer:
"""
prompt = PromptTemplate.from_template(strip_prompt(prompt_text))

In [13]:
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0.1)

In [14]:
chain = (
    {
        "context_txt": retriever_txt,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
question = "가공기의 공구길이를 보정하는 방법을 알려줘, 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

가공기의 공구길이를 보정하는 방법은 다음과 같습니다:

1. 측정하고자 하는 공구를 선택합니다.
2. 수동모드(HANDLE)를 선택하고 Z축을 내려 기준 BLOCK(또는 셋팅바)에 접촉시킵니다.
3. 아래의 방법으로 공구 길이를 계산한 후 그 값을 공구의 옵셋값 길이에 입력합니다.
   - 공구길이 (C) = A - B - D
     - A: 기계 원점으로부터 테이블 상면까지의 거리
     - B: 접촉시 Z축 기계 좌표치
     - D: 기준블럭 길이
4. 측정 완료 후 Z축은 원점복귀 합니다.

이 방법은 하나의 예일 뿐이며 사용자나 제조사에 따라 상이할 수 있으므로 주의하여 실행해야 합니다.

이 정보는 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf' 문서의 38페이지를 참고하였습니다.


In [16]:
# 그림을 보고 답해야하는 질문
question = "가공기 내외부 구성 요소를 알려줘, 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

가공기의 외부 구성 요소에 대한 정보는 다음 문서에서 찾을 수 있습니다:

- 문서 이름: 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf'
- 페이지 번호: 7
- 내용: KT 420 / 420L의 외형도는 일부 옵션을 부착하지 않은 상태로 제공됩니다.

내부 구성 요소에 대한 정보는 다음 문서에서 찾을 수 있습니다:

- 문서 이름: 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf'
- 페이지 번호: 8
- 내용: 내부 구조 및 축 방향에 대한 설명이 포함되어 있습니다.


In [17]:
question = "캘리브레이션이 무엇인지도 설명해줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

캘리브레이션이란 카메라 화면상의 물체 위치 정보 변화량과 로봇의 실제 이동 거리 간의 상관관계를 정의하고, 카메라 화면상에서 1픽셀의 차이가 실제 로봇에서는 얼마나 움직여야 하는지를 정의하는 과정입니다. 이 정보는 문서 'DATA2. e-F@ctory Model Line_Robot-Vision간 모델링 및 캘리브레이션 방법.pdf'의 4페이지를 참고하였습니다.


In [18]:
question = "로봇과 비전간의 캘리브레이션을 어떻게 하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

로봇과 비전 간의 캘리브레이션은 카메라 화면상의 물체 위치 정보 변화량과 로봇의 실제 이동 거리 간의 상관관계를 정의하는 과정입니다. 이 과정에서 카메라 화면상에서 1픽셀의 차이가 실제 로봇에서는 얼마나 움직여야 하는지를 정의합니다. 예를 들어, 1픽셀은 0.026cm에 해당합니다.

이 정보는 'DATA2. e-F@ctory Model Line_Robot-Vision간 모델링 및 캘리브레이션 방법.pdf' 문서의 4페이지에서 참고하였습니다.


In [19]:
question = "AMR 접속이 끊겼을때, IP주소 확인과 재할당 방법에 대해서 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

AMR 접속이 끊겼을 때 IP 주소 확인 방법은 다음과 같습니다:

1. AGV 독립 WiFi에 접속합니다.
2. `mir.com`에 접속합니다.
3. ID는 `admin`을 사용하고, 비밀번호는 입력하지 않습니다.
4. 메뉴에서 `System` -> `System` -> `WiFi 설정`으로 이동합니다.

IP 주소 재할당 방법에 대한 정보는 제공된 문서에서 찾을 수 없습니다.

이 정보는 'DATA3. AMR 접속방법.pdf' 문서의 1페이지를 참고했습니다.


In [20]:
question = "AMR 이동은 가능한데, 도킹 포지션에서 도킹일 안될 경우 해결 방법을 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

도킹 포지션에서 도킹이 안될 경우 해결 방법은 다음과 같습니다:

1. AMR을 방향을 맞춰 충전기 20cm 내에 배치합니다.
2. AMR 설정 웹페이지에 접속합니다.
3. AMR의 충전기를 클릭합니다.
4. Edit를 클릭합니다. (5번 실행 전 값을 확인하기 위해 사진을 찍어두세요.)
5. Detect market을 클릭하여 AMR이 충전기 위치를 재조정합니다.

이 정보는 'DATA5. AMR 충전 실패 조치 방법.pdf' 문서의 3페이지를 참고했습니다.


In [21]:
question = "AMR이 충전기 도킹이 되었는데, 충전에 실패할 경우 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

AMR이 충전기 도킹이 되었지만 충전에 실패할 경우, 다음과 같은 조치를 취할 수 있습니다:

1. AMR을 방향을 맞춰 충전기 20cm 내에 배치합니다.
2. AMR 설정 웹페이지에 접속합니다.
3. AMR의 충전기를 클릭합니다.
4. Edit를 클릭합니다. (이전에 값을 확인하기 위해 사진을 찍어두는 것이 좋습니다.)
5. Detect market를 클릭하여 AMR이 충전기 위치를 재조정합니다.

이 정보는 'DATA5. AMR 충전 실패 조치 방법.pdf' 문서의 2페이지와 3페이지를 참고하였습니다.


In [22]:
question = "레이저 장비에 정면 도어열림 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 정면 도어열림 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 에어가 공급되지 않아 도어가 열리지 않는 경우 에어 공급을 확인합니다.
2. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
3. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
4. 도어 열림 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 11페이지를 참고하였습니다.


In [23]:
question = "레이저 장비에 정면 도어닫힘 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 정면 도어닫힘 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 에어가 공급되지 않아 도어가 닫히지 않는 경우 에어 공급을 확인합니다.
2. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
3. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
4. 도어 열림 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 11페이지를 참고하였습니다.


In [24]:
question = "레이저 장비에 컷팅 클램프 ON 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

컷팅 클램프 ON 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 워크가 공급되지 않거나 위치 이상으로 실린더의 마그네트 센서가 감지 불량이 아닌지 확인합니다.
2. 에어가 공급되지 않아 클램프가 동작되지 않는 경우 에어 공급을 확인합니다.
3. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
4. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
5. 컷팅 클램프 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 11페이지를 참고하였습니다.


In [25]:
# 좀 막연한 질문이 들어온 경우
question = "기계 크기는?"
response = chain.invoke(question)
print(response)

기계 크기는 다음과 같습니다:

- KT420: 높이 2655mm, 소요 바닥 면적 1760×2520mm
- KT420L: 높이 2655mm, 소요 바닥 면적 2064×2520mm

(출처: 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf', 페이지 5)


In [26]:
# 세부 사항에 대한 질문이 들어온 경우
question = "가공기 소요 동력을 세부적으로 설명해줘"
response = chain.invoke(question)
print(response)

가공기의 소요 동력에 대한 세부 사항은 다음과 같습니다:

- **전원**: 220V±10%
- **정격 전원 용량**: 15.6 kVA
- **공압**: 5 kg/㎠ 이상

이 정보는 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf'의 5페이지에서 제공되었습니다. 추가적인 정보가 필요하다면 파일 이름을 참조하여 더 많은 세부 사항을 확인할 수 있습니다.


In [27]:
question = "레이저 장비에 마킹 클램프 ON 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 마킹 클램프 ON 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 워크가 공급되지 않거나 위치 이상으로 실린더의 마그네트 센서가 감지 불량이 아닌지 확인합니다.
2. 에어가 공급되지 않아 클램프가 동작되지 않는 경우 에어 공급을 확인합니다.
3. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
4. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
5. 컷팅 클램프 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 12페이지를 참고하였습니다.


In [28]:
question = "레이저 장비에 MOS전진 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 MOS전진 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 에어가 공급되지 않아 클램프가 동작되지 않는 경우 에어 공급을 확인합니다.
2. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
3. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
4. 동작 확인용 실린더 센서가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 12페이지를 참고하였습니다.


In [29]:
question = "레이저 장비에 마킹 데이터 오류 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 마킹 데이터 오류 알람이 발생했을 때는, OPC UA로부터 반복해서 읽어온 마킹 데이터가 2회 이상 상이할 경우 발생합니다. 이 경우에는 해당 오류 발생 시 제작사에 문의하는 것이 해결 방법입니다.

참고한 문서는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf'의 13페이지입니다.


In [30]:
question = "레이저 장비에 측면 또는 후면 도어 열림 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 측면 또는 후면 도어 열림 알람이 발생했을 때는 다음과 같이 해결할 수 있습니다:

- 원점복귀 또는 운전 중 측면 또는 후면 도어를 열 경우에 발생합니다.
- 도어 닫힘 감지용 센서가 이상 없는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 13페이지를 참고하였습니다.


In [31]:
question = "포장기에서 Vision 인식이 제대로 되지 않을 경우 어떻게 대처하면 되는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

포장기에서 Vision 인식이 제대로 되지 않을 경우, 다음과 같이 대처할 수 있습니다:

1. GOT에서 Bypass 모드를 선택합니다.
2. 한 방향으로 15개씩 적재하여 재기동합니다. (봉투 투입구가 작업자 향하게)
3. 근본적인 조치는 비전 프로그램 검토가 필요하므로 HRC 대응 요청이 필요합니다.

이 정보는 'DATA7. Trouble Shooting_200423.pdf' 문서의 1페이지를 참고하였습니다.


In [32]:
question = "레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성되면 어떻게 해야하나요?"
response = chain.invoke(question)
print(response)

레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성될 경우, 다음과 같은 조치를 고려할 수 있습니다:

1. **레이저 출력 조정**: 컷팅 시의 레이저 출력(Laser Power)을 증가시켜 덜 잘려진 부분을 보완할 수 있습니다. 레이저 출력은 [%] 단위로 조정 가능합니다.

2. **이동 속도 조정**: 컷팅 구간에서의 이동 속도(Speed)를 줄여서 레이저가 더 오랜 시간 동안 소재에 작용하도록 할 수 있습니다. 이동 속도는 [mm/sec] 단위로 조정 가능합니다.

3. **컷팅 경로 확인**: 컷팅이 시작되는 X, Y 좌표(Start X, Start Y)와 잘라낼 아크릴 워크의 크기(Size X, Size Y)를 확인하여 정확한 경로로 컷팅이 이루어졌는지 점검합니다.

4. **추가 이동 거리 설정**: 모서리의 확실한 절단을 위해 Extra 파라미터를 조정하여 실제 아크릴 사이즈보다 약간 이동하는 거리를 설정할 수 있습니다.

이러한 조치를 통해 덜 잘려진 조각 문제를 해결할 수 있습니다. 추가적인 문제가 지속될 경우, 장비의 매뉴얼을 참조하거나 기술 지원에 문의하는 것이 좋습니다.


In [33]:
question = "레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성되면 어떻게 해야하나요?  트러블슈팅 가이드를 참고해주세요."
response = chain.invoke(question)
print(response)

제공된 문서에는 레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성될 때의 트러블슈팅 가이드에 대한 정보가 포함되어 있지 않습니다. 추가적인 정보를 얻기 위해서는 관련 파일 이름을 제공해 주시면 감사하겠습니다.


In [34]:
question = "NC장비에서 안착 불량 에러시 대처?"
response = chain.invoke(question)
print(response)

NC 장비에서 안착 불량 에러가 발생했을 때의 대처 방법은 다음과 같습니다:

1. NC 가공기의 문을 열고 Unclamp 하여 아크릴을 정위치에 놓습니다.
2. NC 가공기의 "Cycle Start" 버튼을 눌러서 재시작합니다.
3. NC GOT에서 자동 재기동 버튼을 누릅니다.
